# Hot-to-notebook steps
- Things we still need to do:
    * Modify the prepare.py function to drop the >90% BMI observation

## Summary
1. Initial markdown cell explaining the project and the notebook's role
1. Executive summary
    * Include the creation of the risk-prediction program

# Stroke Prediction - Final Notebook
## Overview
## Executive Summary

# 1. Wrangle
## Wrangle Overview
* Acquired dataset from Kaggle
* Dropped outliers in BMI and gender columns
* Renamed columns, cast columns to appropriate data types, drop unnecessary columns
* Imputed nulls in BMI column using average BMI for age range and gender
* Plotted univariate distributions to check work
* Noticed class imbalance and need for SMOTE
* Split data

## Script Execution

In [ ]:
# put prepare.py script here to acquire, prep data as explained above

## Univariate Distribution Plots

In [ ]:
# put univariate plots here

* Explain decision to split features into one-hot-encoded feature columns

## Need for SMOTE

In [ ]:
# put visualization here for

* Lightly explain how SMOTE will improve model performance

## Split data for Exploration

## Explore
1. Exploration explained in Markdown
    * Explain decision to split features into one-hot-encoded feature columns
    * Explain effect of age column on other features
    * Explain statistical tests to check if features are driving stroke occurence
    * Explain handling of age's effect on features
1. Code execution of normal statistical tests
    * Independence (Chi2) tests between features and target
    * T-Tests to check drivers
    * Sample vizs to explain process
    * Actual vizs on their own (limiting redundancy of explanation)

1. Code execution of age-unweighted statistical tests for driving-feature elimination

# 2. Explore
## Exploration Overview
* Here's what we did in brief for Explore

## Initial Bivariate Visualizations
* Show how age is distributed differently between had_stroke and no_stroke
    * Explain potential effect of age column on other features

## Initial Hypotheses
### Stats Tests - from MVP notebook

### Combined Results

## Further Hypotheses
### Stats Tests - from combination of team members' notebook work
* Explain statistical tests to check if features are driving stroke occurence
* Independence (Chi2) tests between features and target
* T-Tests to check drivers
* Sample vizs to explain process
* Actual vizs on their own (limiting redundancy of explanation)

### Combined Results

## Age-Unweighted Statistical Tests
* Explain handling of age's effect on features

### Stats Tests

### Results - Features Kept, Eliminated

## Model
1. Modeling explained in Markdown
    * Note use of SMOTE
        * May want to note comparison of non-SMOTEd results vs SMOTEd results
    * Explain encoding decisions
        * Keeping age_range and bmi_range as columns
    * Note models used and hyperparameter combinations (maybe just say "we used various ones")
    * Note process: train, then validate, then test
1. Code execution of preparation for modeling using prepare.py
1. Code execution of model build+fit script
1. Code execution of model evaluation
1. Best model from Validate explained in Markdown
    * Note SMOTE
1. ROC Curve explained in Markdown
1. Code execution of ROC Curve
1. Evaluate best model on Test split
1. Results of Test evaluation explained in Markdown

# Model
## Modeling Overview
### Encoding

### Target Isolation

### Scaling

### SMOTE

## Building, Fitting, Evaluating Models
### Model Preparation Script - No SMOTE

### Model Build+Fit+Predict Script - No SMOTE
- Return non-SMOTEd predictions dataframe, keep separate

### Model Preparation Script - with SMOTE

### Model Build+Fit+Predict Script - with SMOTE
- Return SMOTEd predictions dataframe, keep separate

## Model Performance on Train, Validate
### Model Evaluation Script - Comparing non-SMOTEd and SMOTEd Models
- Join the non-SMOTEd and SMOTEd dataframes together with column renaming to identify if model incorporated SMOTE or not
- Print evaluation results of joined dataframe as normal

## Program
1. Risk-calculation program explained in Markdown
    * Takes user inputs for health information
    * Uses best model from the Model stage to make predictions
    * Returns predictions to user
1. Code execution of a sample row from the dataset in the program

# Risk-Calculation Program
## Risk-Calculation Program Overview
### What it aims to do
### Instructions
## Implementation with Sample Input

## Conclusion
1. Note stats test takeaways
1. Note model performance
1. Explain takeaways
1. Conclude

# Conclusion